# Import packages

In [ ]:
import cv2
import csv
import numpy as np
import os
import sklearn
import warnings
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Convolution2D, Cropping2D, ELU, Dropout, Conv2D, BatchNormalization
from sklearn.model_selection import train_test_split

warnings.filterwarnings("always")

# Prepare Training Data

In [ ]:
def prepare_data(csv_path):
    folder_name = csv_path.split("/")[0]
    center_imgs = []
    left_imgs = []
    right_imgs = []
    steer_angles = []
    with open(csv_path) as csv_file:
        reader = csv.reader(csv_file)
        for line in reader:
            #line: center, left, right, steering angle, throttle, break, speed
            center_imgs.append(folder_name + "/" + line[0].strip())
            left_imgs.append(folder_name + "/" + line[1].strip())
            right_imgs.append(folder_name + "/" + line[2].strip())
            steer_angles.append(float(line[3].strip()))
    return (center_imgs, left_imgs, right_imgs, steer_angles)

## Self generate data 0: official data

In [ ]:
csv_path_0 = 'data/driving_log.csv'
center_paths_0, left_paths_0, right_paths_0, steer_angles_0 = prepare_data(csv_path_0)

### Explore data

In [ ]:
center_paths_0[:3]

In [ ]:
left_paths_0[:3]

In [ ]:
right_paths_0[:3]

In [ ]:
steer_angles_0[:3]

## Self generate data 1: normal track

In [ ]:
csv_path_1 = 'new_data/driving_log.csv'
center_paths_1, left_paths_1, right_paths_1, steer_angles_1 = prepare_data(csv_path_1)

In [ ]:
center_paths_1[:3]

In [ ]:
left_paths_1[:3]

In [ ]:
right_paths_1[:3]

In [ ]:
steer_angles_1[:3]

## Self generate data 2: reverse track

In [ ]:
center_paths_2[:3]

In [ ]:
left_paths_2[:3]

In [ ]:
right_paths_2[:3]

In [ ]:
steer_angles_2[:3]

## Merge training data together

In [ ]:
all_paths = []
all_paths.extend(center_paths_0)
all_paths.extend(left_paths_0)
all_paths.extend(right_paths_0)
all_paths.extend(center_paths_1)
all_paths.extend(left_paths_1)
all_paths.extend(right_paths_1)
all_paths.extend(center_paths_2)
all_paths.extend(left_paths_2)
all_paths.extend(right_paths_2)

all_angles = []
correction = 0.2
all_angles.extend(steer_angles_0)
all_angles.extend([angle + correction for angle in steer_angles_0])
all_angles.extend([angle - correction for angle in steer_angles_0])
all_angles.extend(steer_angles_1)
all_angles.extend([angle + correction for angle in steer_angles_1])
all_angles.extend([angle - correction for angle in steer_angles_1])
all_angles.extend(steer_angles_2)
all_angles.extend([angle + correction for angle in steer_angles_2])
all_angles.extend([angle - correction for angle in steer_angles_2])


In [ ]:
len(all_paths)

In [ ]:
len(all_angles)

In [ ]:
samples = list(zip(all_paths, all_angles))

In [ ]:
# Show the samples data steering distribution

import matplotlib.pyplot as plt
%matplotlib inline

def show_steering(y_train):
    max_degree = 25
    degree_per_steering =10
    n_classes = max_degree * degree_per_steering
    plt.hist(y_train, bins = n_classes, histtype='bar', color='blue', rwidth=0.6, label='train')
    plt.show()


In [ ]:
show_steering(all_angles)

# Define Model Architecture

In [ ]:
def resize(img):
    import tensorflow
    return tensorflow.image.resize_images(img, (60, 120))

In [ ]:
def nvidia_car_model():
    """
    Implementation of nvidia autonomous car model
    """

    # Prepreocessing layers
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape = (160, 320, 3)))
    model.add(Cropping2D(cropping = ((50, 20), (0,0))))

    # Main network
    model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))

    return model

# Training

## Create Training set and validation set

In [ ]:
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [ ]:
len(train_samples)

In [ ]:
len(validation_samples)

## Create model

In [ ]:
model = nvidia_car_model()

In [ ]:
model.summary()

## Complile model